In [110]:
import pandas as pd
import numpy as np
import os
from scipy import stats

# visualize
import seaborn as sns
import matplotlib.pyplot as plt

# acquire
from env import host, user, password
from pydataset import data

# TIDY DATA

### 1. Attendance Data

- Load the attendance.csv file and calculate an attendnace percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.


In [165]:
attendance = pd.read_csv('untidy-data/attendance.csv')
attendance

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [166]:
attendance.columns = ['name', '2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
       '2018-01-05', '2018-01-06', '2018-01-07', '2018-01-08']

In [167]:
attendance_melt = attendance.melt(id_vars='name', var_name='date', value_name='attendance')

In [168]:
attendance_melt['attendance_value'] = attendance_melt.attendance

In [169]:
attendance_melt.head()

,name,date,attendance,attendance_value
0,Sally,2018-01-01,P,P
1,Jane,2018-01-01,A,A
2,Billy,2018-01-01,A,A
3,John,2018-01-01,P,P
4,Sally,2018-01-02,T,T


In [170]:
attendance_melt.attendance_value = attendance_melt.attendance_value.replace(['P','A','H','T'],[1, 0, .5, 0.9])
attendance_melt

,name,date,attendance,attendance_value
0,Sally,2018-01-01,P,1.0
1,Jane,2018-01-01,A,0.0
2,Billy,2018-01-01,A,0.0
3,John,2018-01-01,P,1.0
4,Sally,2018-01-02,T,0.9
5,Jane,2018-01-02,P,1.0
6,Billy,2018-01-02,T,0.9
7,John,2018-01-02,T,0.9
8,Sally,2018-01-03,T,0.9
9,Jane,2018-01-03,T,0.9


In [171]:
grade = attendance_melt.pivot_table(index = 'name')
grade.reset_index(inplace = True)

In [172]:
grade

,name,attendance_value
0,Billy,0.5250
1,Jane,0.6875
2,John,0.9125
3,Sally,0.7625


### 2. Coffee Levels

- Read the coffee_levels.csv file.
- Transform the data so that each carafe is in it's own column.
- Is this the best shape for the data?

In [196]:
coffee = pd.read_csv('untidy-data/coffee_levels.csv')
coffee

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291
5,13,x,0.310711
6,14,x,0.507288
7,15,x,0.215043
8,16,x,0.183891
9,17,x,0.391560


In [197]:
coffee_pivot = coffee.pivot_table(index = 'hour', columns='coffee_carafe', aggfunc='sum')

In [198]:
coffee_pivot

coffee_amount                    
coffee_carafe             x         y         z
hour                                           
8                  0.816164  0.189297  0.999264
9                  0.451018  0.521502  0.915990
10                 0.843279  0.023163  0.144928
11                 0.335533  0.235529  0.311495
12                 0.898291  0.017009  0.771947
13                 0.310711  0.997464  0.398520
14                 0.507288  0.058361  0.864464
15                 0.215043  0.144644  0.436364
16                 0.183891  0.544676  0.280621
17                 0.391560  0.594126  0.436677

It is not the best shape of the data for machine learning, but it is a good way to look at it

### 3. Cake Recipes

- Read the cake_recipes.csv data. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.
- Tidy the data as necessary.


In [260]:
cake_recipe = pd.read_csv('untidy-data/cake_recipes.csv')

In [261]:
cake_recipe

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


In [262]:
cake_recipe = cake_recipe.melt(id_vars='recipe:position', var_name='temp')

In [264]:
cake_recipe[['recipe','position']] = cake_recipe['recipe:position'].str.split(':', expand=True)

In [266]:
cake_recipe.drop(columns='recipe:position', inplace=True)

- Which recipe, on average, is the best? **recipe b**


In [348]:
cake_recipe.groupby('recipe').value.mean().nlargest(1)

recipe
b    76.736074
Name: value, dtype: float64

- Which oven temperature, on average, produces the best results? **275**

In [349]:
cake_recipe.groupby('temp').value.mean().nlargest(1)

temp
275    74.886754
Name: value, dtype: float64

- Which combination of recipe, rack position, and temperature gives the best result? **recipe b, bottom rack, 300 degre**es

In [350]:
cake_recipe.value_counts().nlargest(1, keep='first')

temp  value      recipe  position
300   99.248541  b       bottom      1
dtype: int64